In [1]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [2]:
import minsearch

/Users/sethurama/DEV/LM/course-llm-zc/01-intro/minsearch.py:9: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [44]:
# docs_raw

In [6]:
docs = []
for c in docs_raw:
    for d in c['documents']:
        d['course'] = c['course']
        docs.append(d)

In [7]:
docs[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
# Index into search 

In [9]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [10]:
# SELECT * WHERE course = 'data-engineering-zoomcamp'

In [11]:
q = 'the course has already started, can I still enroll?'

In [12]:
index.fit(docs)

In [24]:
boost = {
    'question': 3.0,
    'section': 0.5
}
results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [18]:
q

'the course has already started, can I still enroll?'

In [14]:
from openai import OpenAI

In [15]:
client = OpenAI()

In [19]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {"role": "user", "content": q}
    ])

In [20]:
response.choices[0].message.content

"It depends on the specific course and institution offering it. Some courses allow late enrollment within a certain time frame, often with the approval of the instructor or department. It's best to contact the institution directly or check their website for policies regarding late enrollment. Make sure to inquire about any potential drawbacks, such as missed coursework or shortened deadlines."

In [21]:
# Lets create the prompt


In [39]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE


QUESTION: {question} 

CONTEXT: 
{context}
""".strip()

In [36]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer:  {doc['text']}\n\n"


In [37]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer:  Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer:  Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer:  The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start 

In [42]:
prompt = prompt_template.format(question=q, context=context).strip()
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE


QUESTION: the course has already started, can I still enroll? 

CONTEXT: 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer:  Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer:  Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone proje

In [43]:
response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{"role": "user", "content": prompt}]
)

response.choices[0].message.content

'Yes, you can still enroll after the course has started. You are also eligible to submit the homeworks, but make sure to meet the deadlines for turning in the final projects.'